<a href="https://colab.research.google.com/github/AnuragSingh7292/Remote-sensing-image-retrieval-deep-learning-/blob/main/VggModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# After running, follow the authentication steps


Mounted at /content/drive


In [ ]:
import zipfile
import os

# Define the paths
zip_path = "/content/drive/MyDrive/Images.zip"  # Update with your actual file path
extract_path = "/content/UC_Merced_LandUse"  # Temporary extraction path

# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Verify extraction
print("Extracted categories:", os.listdir(extract_path))  # Should list 21 folders


Extracted categories: ['Images']


Extracting the features using this Vgg 16 and 19 with PCA 32 dim

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_vgg16
from tensorflow.keras.applications.vgg19 import preprocess_input as preprocess_vgg19
from sklearn.decomposition import PCA

# Dataset path
dataset_path = "/content/UC_Merced_LandUse/Images"
save_dir = "/content/drive/MyDrive/NewONeFeature_VGG_Vectors"
os.makedirs(save_dir, exist_ok=True)

# Load VGG16 and VGG19 models with pre-trained weights
vgg16_model = VGG16(weights='imagenet', include_top=True)
vgg19_model = VGG19(weights='imagenet', include_top=True)

# Extract FC6 layer ('fc1')
vgg16_fc6_layer = tf.keras.Model(inputs=vgg16_model.input,
                                 outputs=vgg16_model.get_layer('fc1').output)
vgg19_fc6_layer = tf.keras.Model(inputs=vgg19_model.input,
                                 outputs=vgg19_model.get_layer('fc1').output)

# Function to extract features
def extract_features(model, img_path, preprocess_func, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_func(img_array)
    features = model.predict(img_array, verbose=0)
    return features.flatten()

# Storage lists
vgg16_fc6_features, vgg19_fc6_features, labels = [], [], []

# Read images and extract features
categories = sorted(os.listdir(dataset_path))
for idx, category in enumerate(categories):
    category_path = os.path.join(dataset_path, category)
    if os.path.isdir(category_path):
        for img_file in os.listdir(category_path):
            if img_file.lower().endswith(('.jpg', '.png', '.tif')):
                img_path = os.path.join(category_path, img_file)

                vgg16_fc6_feat = extract_features(vgg16_fc6_layer, img_path, preprocess_vgg16)
                vgg19_fc6_feat = extract_features(vgg19_fc6_layer, img_path, preprocess_vgg19)

                vgg16_fc6_features.append(vgg16_fc6_feat)
                vgg19_fc6_features.append(vgg19_fc6_feat)
                labels.append(idx)

# Convert lists to arrays
vgg16_fc6_features = np.array(vgg16_fc6_features)
vgg19_fc6_features = np.array(vgg19_fc6_features)

np.save(os.path.join(save_dir, "vgg16_fc6_features.npy"), vgg16_fc6_features)
np.save(os.path.join(save_dir, "vgg19_fc6_features.npy"), vgg19_fc6_features)

# Apply PCA to reduce to (2100, 32)
pca_vgg16_fc6 = PCA(n_components=32)
vgg16_fc6_reduced = pca_vgg16_fc6.fit_transform(vgg16_fc6_features)

pca_vgg19_fc6 = PCA(n_components=32)
vgg19_fc6_reduced = pca_vgg19_fc6.fit_transform(vgg19_fc6_features)

# Save reduced features
np.save(os.path.join(save_dir, "vgg16_fc6_32_features.npy"), vgg16_fc6_reduced)
np.save(os.path.join(save_dir, "vgg19_fc6_32_features.npy"), vgg19_fc6_reduced)
np.save(os.path.join(save_dir, "labels.npy"), labels)

print("✅ FC6 features extracted and saved successfully for VGG16 & VGG19! (Shape: (2100, 32))")


553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 14s 0us/step
574710816/574710816 ━━━━━━━━━━━━━━━━━━━━ 14s 0us/step
✅ FC6 features extracted and saved successfully for VGG16 & VGG19! (Shape: (2100, 32))


In [ ]:
print(vgg16_fc6_features.shape)
print(vgg19_fc6_features.shape)

print(vgg16_fc6_reduced.shape)
print(vgg19_fc6_reduced.shape)

(2100, 4096)
(2100, 4096)
(2100, 32)
(2100, 32)
